# YOLOv8 Dataset Preprocessing

## Dataset

> [Teledyne FLIR Free ADAS Thermal Dataset v2](https://adas-dataset-v2.flirconservator.com/#multipartdownloadsection):
> The Teledyne FLIR free starter thermal dataset provides fully annotated thermal and visible spectrum frames for development of object detection neural networks. This data was constructed to encourage research on visible + thermal spectrum sensor fusion algorithms ("RGBT") in order to advance the safety of autonomous vehicles. A total of 26,442 fully-annotated frames are included with 15 different object classes.


> __Baseline Model__: Baseline accuracy for object detection was established using the YOLOX-m neural network designed for 640 X 640 images. Both the RGB and thermal detectors were pre-trained on MSCOCO data ([YOLOX: Exceeding YOLO Series in 2021](https://arxiv.org/abs/2107.08430) and [YOLOX](https://github.com/Megvii-BaseDetection/YOLOX)). The base neural networks were trained on the training set data provided in this dataset and tested on the video test data also provided in this dataset.

In [1]:
from glob import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from tqdm import tqdm

### Dataset Exploration

> Used to directory path and images to be displayed

In [2]:
# read in dataset
images_dataset = glob('./datasets/FLIR_ADAS_v2/video_thermal_test/images/*.jpg')

In [ ]:
print(len(images_dataset))

In [ ]:
# plot multiple random images
ran_gen = np.random.default_rng()

plt.figure(figsize=(16, 14))
plt.suptitle('Thermal Images')
for i in range(12):
    ax = plt.subplot(4, 4, i+1)
    random_index = ran_gen.integers(low=0, high=3748, size=1)
    i = random_index[0]
    img_loc = images_dataset[i]
    img_title = 'video: ' + images_dataset[i][-52:-35]+'\n'+ 'frame: ' + images_dataset[i][-28:-22]+'\n'+ 'id: ' + images_dataset[i][-21:-4]
    image = plt.imread(img_loc)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.title(img_title, fontsize='small')
    plt.axis(False)

### Label Conversion JSON2YOLO


YOLOv8 expects all images to be located in an `images` dir and the txt format annotation in a `labels` folder next to it. The dataset was using a dirname of `data` for all images and had COCO JSON annotations. Make sure `data` is renamed to `images` with correct image paths by running `scripts/flir_to_yolo.py` first. Now I am able to run a conversion to create the `labels` dir in the same directory as the `images` dir:

In [5]:
def make_folders(output_path):
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path)
    return output_path


def convert_bbox_coco2yolo(img_width, img_height, bbox):
    """
    Convert bounding box from COCO  format to YOLO format

    Parameters
    ----------
    img_width : int
        width of image
    img_height : int
        height of image
    bbox : list[int]
        bounding box annotation in COCO format: 
        [top left x position, top left y position, width, height]

    Returns
    -------
    list[float]
        bounding box annotation in YOLO format: 
        [x_center_rel, y_center_rel, width_rel, height_rel]
    """
    
    # YOLO bounding box format: [x_center, y_center, width, height]
    # (float values relative to width and height of image)
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]

In [6]:
def convert_coco_json_to_yolo_txt(output_path, json_file):

    path = make_folders(output_path)

    with open(json_file) as f:
        json_data = json.load(f)

    # write _darknet.labels, which holds names of all classes (one class per line)
    label_file = os.path.join(output_path, "_darknet.labels")
    with open(label_file, "w") as f:
        for category in tqdm(json_data["categories"], desc="Categories"):
            category_name = category["name"]
            f.write(f"{category_name}\n")

    for image in tqdm(json_data["images"], desc="Annotation txt for each iamge"):
        img_id = image["id"]
        img_name = image["file_name"]
        img_width = image["width"]
        img_height = image["height"]

        anno_in_image = [anno for anno in json_data["annotations"] if anno["image_id"] == img_id]
        anno_txt = os.path.join(output_path, img_name.split(".")[0] + ".txt")
        with open(anno_txt, "w") as f:
            for anno in anno_in_image:
                category = anno["category_id"]
                bbox_COCO = anno["bbox"]
                x, y, w, h = convert_bbox_coco2yolo(img_width, img_height, bbox_COCO)
                f.write(f"{category} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

    print("Converting COCO Json to YOLO txt finished!")

#### Create labels for all image datasets from the parent repo

Pass in the annotations.json and locations of labels to be generated. The `labels` dir will need to be placed in the same directory where the `images` dir is stored. Yolo methods depend on both `labels` and `images` to be in the same directory for training, validation, and testing. 

Either run the Multi Dataset Conversion or the Single Dataset conversion.
* `config` is the default directory for dataset .jsons. However, this will need to be changed if `filtered` was used to narrow down a dataset to specific categories. The dataset dir will remain the same, but replace the `config` dir with filtered one if used.

In [37]:
# Set dataset path for where the image datasets are stored
# Config directory where the coco formatted .json files are stored
DATASET_DIR = './datasets/FLIR_ADAS_v2/'
CONFIG_DIR = './config/'

#### Multi Dataset Conversion

In [ ]:
for dir in os.scandir(DATASET_DIR):
    label_dir = DATASET_DIR + dir.name + '/labels'
    config_dir = CONFIG_DIR + dir.name + '_coco.json'
    convert_coco_json_to_yolo_txt(label_dir, config_dir)

#### Single Dataset Conversion

For processing datasets individually. Run if multi-dataset conversion is not used

In [ ]:
# images_thermal_train
convert_coco_json_to_yolo_txt("./datasets/FLIR_ADAS_v2/images_thermal_train/labels", "./config/images_thermal_train_coco.json")

In [ ]:
# images_thermal_val
convert_coco_json_to_yolo_txt("./datasets/FLIR_ADAS_v2/images_thermal_val/labels", "./config/images_thermal_val_coco.json")

In [ ]:
# video_thermal_test
convert_coco_json_to_yolo_txt("./datasets/FLIR_ADAS_v2/video_thermal_test/labels", "./config/video_thermal_test_coco.json")

In [ ]:
# images_rgb_train
convert_coco_json_to_yolo_txt("./datasets/FLIR_ADAS_v2/images_rgb_train/labels", "./config/images_rgb_train_coco.json")

In [ ]:
# images_rgb_val
convert_coco_json_to_yolo_txt("./datasets/FLIR_ADAS_v2/images_rgb_val/labels", "./config/images_rgb_train_coco.json")

In [ ]:
# video_rgb_test
convert_coco_json_to_yolo_txt("./datasets/FLIR_ADAS_v2/video_rgb_test/labels", "./config/video_rgb_test_coco.json")

### Dataset Configuration

The `coco.yaml` file that came with the dataset contained all 80 COCO classes - I removed all classes that were not part of the annotation and assigned new `category_id`'s from `0`-`15` for the 16 categories. If you want to use the configuration files below to train your YOLO model you need to replace the annotations accordingly - check the `./config` folder.

If a filtered .json is used, the value should match the number of categories. 


* `config/data_thermal.yaml`

```yaml
train: ../images_thermal_train/images
val: ../images_thermal_val/images
test: ../video_thermal_test/images

nc: 16
names: [
  'person',
  'bike',
  'car',
  'motor',
  'bus',
  'train',
  'truck',
  'light',
  'hydrant',
  'sign',
  'dog',
  'deer',
  'skateboard',
  'stroller',
  'scooter',
  'other vehicle'
  ]
```

* `config/data_rgb.yaml`

```yaml
train: /opt/app/datasets/images_rgb_train/images
val: /opt/app/datasets/images_rgb_val/images
test: /opt/app/datasets/video_rgb_test/images

nc: 16
names: [
  'person',
  'bike',
  'car',
  'motor',
  'bus',
  'train',
  'truck',
  'light',
  'hydrant',
  'sign',
  'dog',
  'deer',
  'skateboard',
  'stroller',
  'scooter',
  'other vehicle'
  ]
```

#### Multi Annotation Count

In [30]:
# Config directory where the coco formatted .json files are stored
CONFIG_DIR = './config_person/'

In [ ]:
for file in os.listdir(CONFIG_DIR):
    if file.endswith('.json'):
        f_config = open(CONFIG_DIR + file) # => './datasets/images_thermal_val/coco.json'
        # returns JSON object as a dictionary
        config_val = json.load(f_config)
        # closing files
        f_config.close()

        categories = []

        for detection in config_val['annotations']:
            categories.append(detection['category_id'])

        print("{}: {}".format(file, np.unique(categories)))

#### Single Annotation Count

In [32]:
f_rgb = open('./config/images_rgb_val_coco.json') # =>'./datasets/images_rgb_val/coco.json'
f_thermal = open('./config/images_thermal_val_coco.json') # => './datasets/images_thermal_val/coco.json'
# returns JSON object as a dictionary
data_rgb_val = json.load(f_rgb)
data_thermal_val = json.load(f_thermal)
# closing files
f_rgb.close()
f_thermal.close()

In [33]:
f_rgb = open('./config/images_rgb_train_coco.json') # => './datasets/images_rgb_train/coco.json'
f_thermal = open('./config/images_thermal_train_coco.json') # => './datasets/images_thermal_train/coco.json'
# returns JSON object as a dictionary
data_rgb_train = json.load(f_rgb)
data_thermal_train = json.load(f_thermal)
# closing files
f_rgb.close()
f_thermal.close()

In [34]:
f_rgb = open('./config/video_rgb_test_coco.json') # => './datasets/video_rgb_test/coco.json'
f_thermal = open('./config/video_thermal_test_coco.json') # => './datasets/video_thermal_test/coco.json'
# returns JSON object as a dictionary
data_rgb_test = json.load(f_rgb)
data_thermal_test = json.load(f_thermal)
# closing files
f_rgb.close()
f_thermal.close()

In [ ]:
# Iterating through the json list - check that all annotations are between 0 and 15

categories = []

for detection in data_rgb_val['annotations']:
    categories.append(detection['category_id'])

print(np.unique(categories))

In [ ]:
categories = []

for detection in data_thermal_val['annotations']:
    categories.append(detection['category_id'])

print(np.unique(categories))

In [ ]:
categories = []

for detection in data_rgb_train['annotations']:
    categories.append(detection['category_id'])

print(np.unique(categories))

In [ ]:
categories = []

for detection in data_thermal_train['annotations']:
    categories.append(detection['category_id'])

print(np.unique(categories))

In [ ]:
categories = []

for detection in data_rgb_test['annotations']:
    categories.append(detection['category_id'])

print(np.unique(categories))

In [ ]:
categories = []

for detection in data_thermal_test['annotations']:
    categories.append(detection['category_id'])

print(np.unique(categories))